###DBScan (Density-based spatial clustering of applications with noise)

lets start with the basics in  nutshell Clustering is the process of examining a collection of “points,” and grouping the points into “clusters” according to some distance measure. The goal is that points in the same cluster have a small distance from one another, while points in different clusters are at a large distance from one another.

now lets wiki some info
It is a density-based clustering algorithm: given a set of points in some space, it groups together points that are closely packed together (points with many nearby neighbors), marking as outliers points that lie alone in low-density regions (whose nearest neighbors are too far away). DBSCAN is one of the most common clustering algorithms and also most cited in scientific literature.

<p>DBSCAN requires two parameters: ε (eps) and the minimum number of points required to form a dense region<sup id="cite_ref-minpts_5-0" class="reference"><a href="#cite_note-minpts-5">[a]</a></sup> (minPts). It starts with an arbitrary starting point that has not been visited. This point's ε-neighborhood is retrieved, and if it contains sufficiently many points, a cluster is started. Otherwise, the point is labeled as noise. Note that this point might later be found in a sufficiently sized ε-environment of a different point and hence be made part of a cluster.</p>
<p>If a point is found to be a dense part of a cluster, its ε-neighborhood is also part of that cluster. Hence, all points that are found within the ε-neighborhood are added, as is their own ε-neighborhood when they are also dense. This process continues until the density-connected cluster is completely found. Then, a new unvisited point is retrieved and processed, leading to the discovery of a further cluster or noise.</p>
<p>DBSCAN can be used with any distance function<sup id="cite_ref-dbscan_1-1" class="reference"><a href="#cite_note-dbscan-1">[1]</a></sup><sup id="cite_ref-tods_4-2" class="reference"><a href="#cite_note-tods-4">[4]</a></sup> (as well as similarity functions or other predicates).<sup id="cite_ref-:0_6-0" class="reference"><a href="#cite_note-:0-6">[5]</a></sup> The distance function (dist) can therefore be seen as an additional parameter.</p>
<pre>DBSCAN(DB, dist, eps, minPts) {
   C = 0                                              <i>/* Cluster counter */</i>
   <b>for each</b> point P <b>in</b> database DB {
      <b>if</b> label(P) ≠ undefined <b>then</b> <b>continue</b>           <i>/* Previously processed in inner loop */</i>
      Neighbors N = RangeQuery(DB, dist, P, eps)      <i>/* Find neighbors */</i>
      <b>if</b> |N| &lt; minPts <b>then</b> {                          <i>/* Density check */</i>
         label(P) = Noise                             <i>/* Label as Noise */</i>
         <b>continue</b>
      }
      C = C + 1                                       <i>/* next cluster label */</i>
      label(P) = C                                    <i>/* Label initial point */</i>
      Seed set S = N \ {P}                            <i>/* Neighbors to expand */</i>
      <b>for each</b> point Q <b>in</b> S {                         <i>/* Process every seed point */</i>
         <b>if</b> label(Q) = Noise <b>then</b> label(Q) = C        <i>/* Change Noise to border point */</i>
         <b>if</b> label(Q) ≠ undefined <b>then</b> <b>continue</b>        <i>/* Previously processed */</i>
         label(Q) = C                                 <i>/* Label neighbor */</i>
         Neighbors N = RangeQuery(DB, dist, Q, eps)   <i>/* Find neighbors */</i>
         <b>if</b> |N| ≥ minPts <b>then</b> {                       <i>/* Density check */</i>
            S = S ∪ N                                 <i>/* Add new neighbors to seed set */</i>
         }
      }
   }
}
</pre>
<pre>RangeQuery(DB, dist, Q, eps) {
   Neighbors = empty list
   <b>for each</b> point P <b>in</b> database DB {                  <i>/* Scan all points in the database */</i>
      <b>if</b> dist(Q, P) ≤ eps <b>then</b> {                      <i>/* Compute distance and check epsilon */</i>
         Neighbors = Neighbors ∪ {P}                  <i>/* Add to result */</i>
      }
   }
   <b>return</b> Neighbors
}
</pre>

<h3>Advantages</h3>
<ol>
<li>DBSCAN does not require one to specify the number of clusters in the data a priori, as opposed to <a href="/wiki/K-means_algorithm" class="mw-redirect" title="K-means algorithm">k-means</a>.</li>
<li>DBSCAN can find arbitrarily shaped clusters. It can even find a cluster completely surrounded by (but not connected to) a different cluster. Due to the MinPts parameter, the so-called single-link effect (different clusters being connected by a thin line of points) is reduced.</li>
<li>DBSCAN has a notion of noise, and is robust to <a href="/wiki/Anomaly_detection" title="Anomaly detection">outliers</a>.</li>
<li>DBSCAN requires just two parameters and is mostly insensitive to the ordering of the points in the database. (However, points sitting on the edge of two different clusters might swap cluster membership if the ordering of the points is changed, and the cluster assignment is unique only up to isomorphism.)</li>
<li>DBSCAN is designed for use with databases that can accelerate region queries, e.g. using an <a href="/wiki/R*_tree" title="R* tree">R* tree</a>.</li>
<li>The parameters minPts and ε can be set by a domain expert, if the data are well understood.</li>
</ol>
<h3>Disadvantages</h3>
<ol>
<li>DBSCAN is not entirely deterministic: border points that are reachable from more than one cluster can be part of either cluster, depending on the order the data are processed. For most data sets and domains, this situation fortunately does not arise often and has little impact on the clustering result:<sup id="cite_ref-tods_4-5" class="reference"><a href="#cite_note-tods-4">[4]</a></sup> both on core points and noise points, DBSCAN is deterministic. DBSCAN*<sup id="cite_ref-hdbscan1_7-0" class="reference"><a href="#cite_note-hdbscan1-7">[6]</a></sup> is a variation that treats border points as noise, and this way achieves a fully deterministic result as well as a more consistent statistical interpretation of density-connected components.</li>
<li>The quality of DBSCAN depends on the <a href="/wiki/Metric_(mathematics)" title="Metric (mathematics)">distance measure</a> used. The most common distance metric used is <a href="/wiki/Euclidean_distance" title="Euclidean distance">Euclidean distance</a>. Especially for <a href="/wiki/Clustering_high-dimensional_data" title="Clustering high-dimensional data">high-dimensional data</a>, this metric can be rendered almost useless due to the so-called "<a href="/wiki/Curse_of_dimensionality#Distance_functions" title="Curse of dimensionality">Curse of dimensionality</a>", making it difficult to find an appropriate value for ε. This effect, however, is also present in any other algorithm based on Euclidean distance.</li>
<li>DBSCAN cannot cluster data sets well with large differences in densities, since the minPts-ε combination cannot then be chosen appropriately for all clusters.</li>
<li>If the data and scale are not well understood, choosing a meaningful distance threshold ε can be difficult.</li>
</ol>

The density of a data point is defined by the number of points that lie within a radius
Eps of that point (including the point itself). The densities of these spherical regions are
used to classify the data points into core, border, or noise points. These notions are defined
as follows:
1. Core point: A data point is defined as a core point, if it contains4 at least τ data
points.
2. Border point: A data point is defined as a border point, if it contains less than τ points,
but it also contains at least one core point within a radius Eps.
3. Noise point: A data point that is neither a core point nor a border point is defined as
a noise point.

The DBSCAN method can discover clusters of
arbitrary shape, and it does not require the number of clusters as an input parameter. As in
the case of grid-based methods, it is susceptible to variations in the local cluster density. For
example, in Figs. 6.4b and 6.14, DBSCAN will either not discover the sparse cluster, or it
might merge the two dense clusters. In such cases, algorithms such as Mahalanobis k-means
are more effective because of their ability to normalize the distances with local density. On
the other hand, DBSCAN will be able to effectively discover the clusters of Fig. 6.4a, which
is not possible with the Mahalanobis k-means method.
The major time complexity of DBSCAN is in finding the neighbors of the different
data points within a distance of Eps. For a database of size n, the time complexity can be
O(n2) in the worst case. However, for some special cases, the use of a spatial index for finding
the nearest neighbors can reduce this to approximately O(n·log(n)) distance computations.
The O(log(n)) query performance is realized only for low-dimensional data, in which nearest
neighbor indexes work well. In general, grid-based methods are more efficient because they
partition the space, rather than opting for the more computationally intensive approach of
finding the nearest neighbors.
The parameters τ and Eps are related to one another in an intuitive way, which is
useful for parameter setting. In particular, after the value of τ has been set by the user,
the value of Eps can be determined in a data-driven way. The idea is to use a value of Eps
that can capture most of the data points in clusters as core points. This can be achieved
as follows. For each data point, its τ -nearest neighbor distance is determined. Typically,
the vast majority of the data points inside clusters will have a small value of the τ -nearest
neighbor distance. However, the value of the τ -nearest neighbor often increases suddenly
for a small number of noisy points (or points at the fringes of clusters). Therefore, the key
is to identify the tail of the distribution of τ -nearest neighbor distances. Statistical tests,
such as the Z-value test, can be used in order to determine the value of Eps at which the
τ -nearest neighbor distance starts increasing abruptly. This value of the τ -nearest neighbor
distance at this cutoff point provides a suitable value of Eps.

Clustering is the process of examining a collection of “points,” and grouping
the points into “clusters” according to some distance measure. The goal is that
points in the same cluster have a small distance from one another, while points
in different clusters are at a large distance from one another